In [26]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import sys
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers

class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

### Buy and HODL
class BaselineStrategy(bt.Strategy):
    
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            cash = self.broker.get_cash()
            limit_price = self.data.close[0] * 1.01
            size = (cash * 0.95) / limit_price
            print(cash, self.data.close[0], limit_price, size)
            self.buy(exectype=bt.Order.Limit, size=size)

cerebro = bt.Cerebro()

from_date=datetime.datetime(2016, 1, 1)
to_date = datetime.datetime(2020, 12, 30)

# Create data feeds
data_ticks = bt.feeds.CCXT(exchange='coinbasepro', symbol='BTC/USD', name="btc_usd_tick",
                           timeframe=bt.TimeFrame.Days, compression=1, fromdate=from_date, todate=to_date)
cerebro.adddata(data_ticks)

cerebro.broker.setcash(10000.0)
cerebro.broker.addcommissioninfo(CommInfoFractional())
cerebro.broker.setcommission(commission=0.005, margin=False, mult=1.0, commtype=None, percabs=True, stocklike=False, interest=0.0, interest_long=False, leverage=1.0, automargin=False, name=None)

cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='tradeanalyzer')

# cerebro.addobserver(bt.observers.DrawDown)
# cerebro.addobserver(bt.observers.LogReturns)
# cerebro.addobserver(bt.observers.TimeReturn)

# Add the strategy
cerebro.addstrategy(BaselineStrategy)

# Run the strategy
strats = cerebro.run()

2016-01-01, Close, 435.66
10000.0 435.66 440.01660000000004 21.590094555523585
2016-01-02, BUY EXECUTED, 435.66
2016-01-02, Close, 435.40
2016-01-03, Close, 431.91
2016-01-04, Close, 433.85
2016-01-05, Close, 433.34
2016-01-06, Close, 430.87
2016-01-07, Close, 459.07
2016-01-08, Close, 454.44
2016-01-09, Close, 450.38
2016-01-10, Close, 449.99
2016-01-11, Close, 449.19
2016-01-12, Close, 434.01
2016-01-13, Close, 432.77
2016-01-14, Close, 430.03
2016-01-15, Close, 357.53
2016-01-16, Close, 388.70
2016-01-17, Close, 378.46
2016-01-18, Close, 384.89
2016-01-19, Close, 375.27
2016-01-20, Close, 418.54
2016-01-21, Close, 409.38
2016-01-22, Close, 382.90
2016-01-23, Close, 387.50
2016-01-24, Close, 403.05
2016-01-25, Close, 391.40
2016-01-26, Close, 391.54
2016-01-27, Close, 394.79
2016-01-28, Close, 379.61
2016-01-29, Close, 378.68
2016-01-30, Close, 378.46
2016-01-31, Close, 367.95
2016-02-01, Close, 371.33
2016-02-02, Close, 372.93
2016-02-03, Close, 368.87
2016-02-04, Close, 387.99
2016

2017-01-01, Close, 992.95
2017-01-02, Close, 1011.45
2017-01-03, Close, 1020.67
2017-01-04, Close, 1130.30
2017-01-05, Close, 1007.00
2017-01-06, Close, 895.71
2017-01-07, Close, 909.00
2017-01-08, Close, 923.33
2017-01-09, Close, 902.66
2017-01-10, Close, 907.00
2017-01-11, Close, 795.77
2017-01-12, Close, 812.25
2017-01-13, Close, 831.42
2017-01-14, Close, 828.00
2017-01-15, Close, 833.31
2017-01-16, Close, 834.41
2017-01-17, Close, 906.00
2017-01-18, Close, 887.99
2017-01-19, Close, 902.92
2017-01-20, Close, 894.31
2017-01-21, Close, 925.06
2017-01-22, Close, 931.22
2017-01-23, Close, 910.70
2017-01-24, Close, 887.00
2017-01-25, Close, 896.00
2017-01-26, Close, 917.02
2017-01-27, Close, 922.63
2017-01-28, Close, 924.98
2017-01-29, Close, 916.54
2017-01-30, Close, 923.23
2017-01-31, Close, 970.01
2017-02-01, Close, 992.75
2017-02-02, Close, 1008.38
2017-02-03, Close, 1018.00
2017-02-04, Close, 1035.54
2017-02-05, Close, 1016.32
2017-02-06, Close, 1022.58
2017-02-07, Close, 1053.96
20

2018-03-13, Close, 9145.41
2018-03-14, Close, 8207.02
2018-03-15, Close, 8259.99
2018-03-16, Close, 8275.00
2018-03-17, Close, 7857.60
2018-03-18, Close, 8192.00
2018-03-19, Close, 8589.00
2018-03-20, Close, 8900.00
2018-03-21, Close, 8891.81
2018-03-22, Close, 8715.09
2018-03-23, Close, 8927.10
2018-03-24, Close, 8531.34
2018-03-25, Close, 8453.00
2018-03-26, Close, 8145.00
2018-03-27, Close, 7793.61
2018-03-28, Close, 7942.72
2018-03-29, Close, 7080.00
2018-03-30, Close, 6848.01
2018-03-31, Close, 6928.50
2018-04-01, Close, 6816.01
2018-04-02, Close, 7045.01
2018-04-03, Close, 7424.90
2018-04-04, Close, 6791.68
2018-04-05, Close, 6785.85
2018-04-06, Close, 6619.01
2018-04-07, Close, 6894.01
2018-04-08, Close, 7020.01
2018-04-09, Close, 6771.13
2018-04-10, Close, 6824.99
2018-04-11, Close, 6942.99
2018-04-12, Close, 7916.00
2018-04-13, Close, 7893.19
2018-04-14, Close, 8003.11
2018-04-15, Close, 8355.25
2018-04-16, Close, 8048.93
2018-04-17, Close, 7892.10
2018-04-18, Close, 8152.05
2

2019-08-14, Close, 10025.86
2019-08-15, Close, 10300.01
2019-08-16, Close, 10352.78
2019-08-17, Close, 10217.79
2019-08-18, Close, 10315.48
2019-08-19, Close, 10920.00
2019-08-20, Close, 10769.09
2019-08-21, Close, 10130.00
2019-08-22, Close, 10107.15
2019-08-23, Close, 10410.85
2019-08-24, Close, 10147.96
2019-08-25, Close, 10138.72
2019-08-26, Close, 10361.65
2019-08-27, Close, 10171.85
2019-08-28, Close, 9714.31
2019-08-29, Close, 9495.00
2019-08-30, Close, 9582.42
2019-08-31, Close, 9600.86
2019-09-01, Close, 9766.52
2019-09-02, Close, 10381.26
2019-09-03, Close, 10628.48
2019-09-04, Close, 10581.84
2019-09-05, Close, 10574.73
2019-09-06, Close, 10308.92
2019-09-07, Close, 10484.37
2019-09-08, Close, 10398.97
2019-09-09, Close, 10310.00
2019-09-10, Close, 10093.01
2019-09-11, Close, 10160.20
2019-09-12, Close, 10423.72
2019-09-13, Close, 10365.93
2019-09-14, Close, 10360.25
2019-09-15, Close, 10304.46
2019-09-16, Close, 10262.54
2019-09-17, Close, 10185.39
2019-09-18, Close, 10155.

2020-06-28, Close, 9113.24
2020-06-29, Close, 9184.45
2020-06-30, Close, 9136.20
2020-07-01, Close, 9239.97
2020-07-02, Close, 9091.53
2020-07-03, Close, 9063.87
2020-07-04, Close, 9138.51
2020-07-05, Close, 9074.39
2020-07-06, Close, 9343.31
2020-07-07, Close, 9253.94
2020-07-08, Close, 9435.28
2020-07-09, Close, 9236.95
2020-07-10, Close, 9291.05
2020-07-11, Close, 9235.00
2020-07-12, Close, 9303.37
2020-07-13, Close, 9236.92
2020-07-14, Close, 9256.94
2020-07-15, Close, 9190.16
2020-07-16, Close, 9130.11
2020-07-17, Close, 9154.29
2020-07-18, Close, 9175.85
2020-07-19, Close, 9212.87
2020-07-20, Close, 9161.05
2020-07-21, Close, 9394.14
2020-07-22, Close, 9536.18
2020-07-23, Close, 9613.24
2020-07-24, Close, 9551.21
2020-07-25, Close, 9711.33
2020-07-26, Close, 9941.00
2020-07-27, Close, 11048.91
2020-07-28, Close, 10934.23
2020-07-29, Close, 11110.12
2020-07-30, Close, 11115.95
2020-07-31, Close, 11351.62
2020-08-01, Close, 11810.07
2020-08-02, Close, 11068.54
2020-08-03, Close, 11

In [24]:
print('Returns:', strats[0].analyzers.returns.pprint())
print('Sharpe Ratio:', strats[0].analyzers.mysharpe.pprint())
print('Annual Return:', strats[0].analyzers.annualreturn.pprint())
print('Drawdown:', strats[0].analyzers.drawdown.pprint())
print('Trade Analyzer:', strats[0].analyzers.tradeanalyzer.pprint())
cerebro.plot()

OrderedDict([('rtot', 4.1342782581838255),
             ('ravg', 0.0022641173374500687),
             ('rnorm', 0.7692532573048794),
             ('rnorm100', 76.92532573048794)])
Returns: None
OrderedDict([('sharperatio', 0.7075709507425372)])
Sharpe Ratio: None
OrderedDict([(2016, 1.156218004048029),
             (2017, 12.906447153096254),
             (2018, -0.7323536823676501),
             (2019, 0.9345374851152528),
             (2020, 3.0220452700108593)])
Annual Return: None
AutoOrderedDict([('len', 0),
                 ('drawdown', 0.0),
                 ('moneydown', 0.0),
                 ('max',
                  AutoOrderedDict([('len', 1079),
                                   ('drawdown', 83.69361893711935),
                                   ('moneydown', 355524.3029467524)]))])
Drawdown: None
AutoOrderedDict([('total', AutoOrderedDict([('total', 1), ('open', 1)]))])
Trade Analyzer: None


TypeError: list indices must be integers or slices, not NoneType